<a href="https://colab.research.google.com/github/zsowen/Project_3/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install vaderSentiment

In [2]:
# Import the libraries
import pandas as pd
import numpy as np
from textblob import TextBlob
import re
import nltk
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [3]:
# load data
from google.colab import files
files.upload()

Saving needsPrices.xlsx to needsPrices (1).xlsx
Saving withPrices.xlsx to withPrices (1).xlsx


In [38]:
# load data
from google.colab import files
files.upload()

Saving withPricesAndChange.xlsx to withPricesAndChange.xlsx


In [17]:
# store the data as a variable
needsPrices_df= pd.read_excel('needsPrices.xlsx')
# 
withPrices_df= pd.read_excel('withPrices.xlsx')


In [39]:
#
Prices_with_changes = pd.read_excel('withPricesAndChange.xlsx')

In [78]:
# peak at data
Prices_with_changes.head(3)

,title,Symbol,Security,Year,Month,Day,Date,Key,Opening,Closing,StockChg,WeekNo
0,Agilent to Buy Multiplicom for ‚Ç¨68M in Cash,A,Agilent Technologies,2016,12,20,2016-12-20,2016-12-20A,45.750000,46.209999,Positive,52
1,Pressure Biosciences CEO On Its Beaten Down St...,A,Agilent Technologies,2016,12,6,2016-12-06,2016-12-06A,44.580002,44.840000,Positive,50
2,"The Market In 5 Minutes: DryShips, Trump And T...",A,Agilent Technologies,2016,11,16,2016-11-16,2016-11-16A,46.299999,46.180000,Negative,47


In [79]:
Prices_with_changes.head(10)

,title,Symbol,Security,Year,Month,Day,Date,Key,Opening,Closing,StockChg,WeekNo
0,Agilent to Buy Multiplicom for ‚Ç¨68M in Cash,A,Agilent Technologies,2016,12,20,2016-12-20,2016-12-20A,45.750000,46.209999,Positive,52
1,Pressure Biosciences CEO On Its Beaten Down St...,A,Agilent Technologies,2016,12,6,2016-12-06,2016-12-06A,44.580002,44.840000,Positive,50
2,"The Market In 5 Minutes: DryShips, Trump And T...",A,Agilent Technologies,2016,11,16,2016-11-16,2016-11-16A,46.299999,46.180000,Negative,47
3,A Peek Into The Markets: U.S. Stock Futures De...,A,Agilent Technologies,2016,11,16,2016-11-16,2016-11-16A,46.299999,46.180000,Negative,47
4,"Keep an Eye on These 8 Stocks for November 16,...",A,Agilent Technologies,2016,11,16,2016-11-16,2016-11-16A,46.299999,46.180000,Negative,47
5,Agilent Technologies Sees FY17 Rev. $4.35B-$4....,A,Agilent Technologies,2016,11,15,2016-11-15,2016-11-15A,45.330002,45.400002,Positive,47
6,Agilent Technologies Reports Q4 EPS $0.59 vs. ...,A,Agilent Technologies,2016,11,15,2016-11-15,2016-11-15A,45.330002,45.400002,Positive,47
7,8 Stocks You Should Be Watching Today,A,Agilent Technologies,2016,11,15,2016-11-15,2016-11-15A,45.330002,45.400002,Positive,47
8,"Earnings Scheduled For November 15, 2016",A,Agilent Technologies,2016,11,15,2016-11-15,2016-11-15A,45.330002,45.400002,Positive,47
9,"Leerink Swann Assumes Agilent Technologies, In...",A,Agilent Technologies,2016,11,10,2016-11-10,2016-11-10A,46.599998,47.029999,Positive,46


In [80]:
A_ticker= Prices_with_changes.loc[Prices_with_changes['Symbol'] == 'A']
A_ticker

,title,Symbol,Security,Year,Month,Day,Date,Key,Opening,Closing,StockChg,WeekNo
0,Agilent to Buy Multiplicom for ‚Ç¨68M in Cash,A,Agilent Technologies,2016,12,20,2016-12-20,2016-12-20A,45.750000,46.209999,Positive,52
1,Pressure Biosciences CEO On Its Beaten Down St...,A,Agilent Technologies,2016,12,6,2016-12-06,2016-12-06A,44.580002,44.840000,Positive,50
2,"The Market In 5 Minutes: DryShips, Trump And T...",A,Agilent Technologies,2016,11,16,2016-11-16,2016-11-16A,46.299999,46.180000,Negative,47
3,A Peek Into The Markets: U.S. Stock Futures De...,A,Agilent Technologies,2016,11,16,2016-11-16,2016-11-16A,46.299999,46.180000,Negative,47
4,"Keep an Eye on These 8 Stocks for November 16,...",A,Agilent Technologies,2016,11,16,2016-11-16,2016-11-16A,46.299999,46.180000,Negative,47
...,...,...,...,...,...,...,...,...,...,...,...,...
573,The Health Care Reform Bill: Welcome To The Bi...,A,Agilent Technologies,2010,3,11,2010-03-11,2010-03-11A,33.379999,33.580001,Positive,11
574,BRKR Buys VARI‚Äôs Assets; Deal To Be Accretiv...,A,Agilent Technologies,2010,3,10,2010-03-10,2010-03-10A,33.199999,33.660000,Positive,11
575,The Google Buzz Guide for the Enterprise (with...,A,Agilent Technologies,2010,2,17,2010-02-17,2010-02-17A,30.359999,30.630001,Positive,8
576,Agilent Technologies (A) Reports 23% Profit Rise,A,Agilent Technologies,2010,2,12,2010-02-12,2010-02-12A,30.650000,29.980000,Negative,7


In [81]:
A_ticker= A_ticker.iloc[::-1]

In [82]:
A_ticker= A_ticker.reset_index()
A_ticker

,index,title,Symbol,Security,Year,Month,Day,Date,Key,Opening,Closing,StockChg,WeekNo
0,577,"Benzinga‚Äôs Earnings Roundup (IR, NWE, HCP, A)",A,Agilent Technologies,2010,2,12,2010-02-12,2010-02-12A,30.650000,29.980000,Negative,7
1,576,Agilent Technologies (A) Reports 23% Profit Rise,A,Agilent Technologies,2010,2,12,2010-02-12,2010-02-12A,30.650000,29.980000,Negative,7
2,575,The Google Buzz Guide for the Enterprise (with...,A,Agilent Technologies,2010,2,17,2010-02-17,2010-02-17A,30.359999,30.630001,Positive,8
3,574,BRKR Buys VARI‚Äôs Assets; Deal To Be Accretiv...,A,Agilent Technologies,2010,3,10,2010-03-10,2010-03-10A,33.199999,33.660000,Positive,11
4,573,The Health Care Reform Bill: Welcome To The Bi...,A,Agilent Technologies,2010,3,11,2010-03-11,2010-03-11A,33.379999,33.580001,Positive,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...
573,4,"Keep an Eye on These 8 Stocks for November 16,...",A,Agilent Technologies,2016,11,16,2016-11-16,2016-11-16A,46.299999,46.180000,Negative,47
574,3,A Peek Into The Markets: U.S. Stock Futures De...,A,Agilent Technologies,2016,11,16,2016-11-16,2016-11-16A,46.299999,46.180000,Negative,47
575,2,"The Market In 5 Minutes: DryShips, Trump And T...",A,Agilent Technologies,2016,11,16,2016-11-16,2016-11-16A,46.299999,46.180000,Negative,47
576,1,Pressure Biosciences CEO On Its Beaten Down St...,A,Agilent Technologies,2016,12,6,2016-12-06,2016-12-06A,44.580002,44.840000,Positive,50


In [83]:
# Create a function to get the subjectivity
def getSubjectivity(text):
  return TextBlob(text).sentiment.subjectivity

In [84]:
# Create a function to get the polarity
def getPolarity(text):
  return  TextBlob(text).sentiment.polarity

In [85]:
# Create two new columns 'Subjectivity' & 'Polarity'
A_ticker['Subjectivity'] =A_ticker['title'].apply(getSubjectivity)
A_ticker['Polarity'] =A_ticker['title'].apply(getPolarity)

In [86]:
#Create a function to get the sentiment scores (using Sentiment Intensity Analyzer)
def getSIA(text):
  sia = SentimentIntensityAnalyzer()
  sentiment = sia.polarity_scores(text)
  return sentiment


In [87]:
# The compound score is a metric that calculates the sum of all the lexicon
#  ratings which have been normalized between -1(most extreme negative)
#  and +1 (most extreme positive).

In [88]:
#Get the sentiment scores for each day
compound = []
neg = []
neu = []
pos = []
SIA = 0
for i in range(0, len(A_ticker['title'])):
  SIA = getSIA(A_ticker['title'][i])
  compound.append(SIA['compound'])
  neg.append(SIA['neg'])
  neu.append(SIA['neu'])
  pos.append(SIA['pos'])

In [89]:
#Store the sentiment scores in the data frame
A_ticker['Compound'] =compound
A_ticker['Negative'] =neg
A_ticker['Neutral'] =neu
A_ticker['Positive'] = pos

In [97]:
A_ticker= A_ticker.round(4)
A_ticker.loc[(A_ticker.StockChg == 'Positive'), 'StockChg'] = 1
A_ticker.loc[(A_ticker.StockChg == 'Negative'), 'StockChg'] = 0
A_ticker["StockChg"] = pd.to_numeric(A_ticker["StockChg"])
A_ticker.dtypes

index                    int64
title                   object
Symbol                  object
Security                object
Year                     int64
Month                    int64
Day                      int64
Date            datetime64[ns]
Key                     object
Opening                float64
Closing                float64
StockChg                 int64
WeekNo                   int64
Subjectivity           float64
Polarity               float64
Compound               float64
Negative               float64
Neutral                float64
Positive               float64
dtype: object

In [98]:
A_ticker

,index,title,Symbol,Security,Year,Month,Day,Date,Key,Opening,Closing,StockChg,WeekNo,Subjectivity,Polarity,Compound,Negative,Neutral,Positive
0,577,"Benzinga‚Äôs Earnings Roundup (IR, NWE, HCP, A)",A,Agilent Technologies,2010,2,12,2010-02-12,2010-02-12A,30.65,29.98,0,7,0.0000,0.0000,0.0000,0.000,1.000,0.000
1,576,Agilent Technologies (A) Reports 23% Profit Rise,A,Agilent Technologies,2010,2,12,2010-02-12,2010-02-12A,30.65,29.98,0,7,0.0000,0.0000,0.4404,0.000,0.674,0.326
2,575,The Google Buzz Guide for the Enterprise (with...,A,Agilent Technologies,2010,2,17,2010-02-17,2010-02-17A,30.36,30.63,1,8,0.0000,0.0000,0.0000,0.000,1.000,0.000
3,574,BRKR Buys VARI‚Äôs Assets; Deal To Be Accretiv...,A,Agilent Technologies,2010,3,10,2010-03-10,2010-03-10A,33.20,33.66,1,11,0.0000,0.0000,0.1779,0.000,0.841,0.159
4,573,The Health Care Reform Bill: Welcome To The Bi...,A,Agilent Technologies,2010,3,11,2010-03-11,2010-03-11A,33.38,33.58,1,11,0.9000,0.8000,0.8176,0.000,0.564,0.436
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
573,4,"Keep an Eye on These 8 Stocks for November 16,...",A,Agilent Technologies,2016,11,16,2016-11-16,2016-11-16A,46.30,46.18,0,47,0.0000,0.0000,0.0000,0.000,1.000,0.000
574,3,A Peek Into The Markets: U.S. Stock Futures De...,A,Agilent Technologies,2016,11,16,2016-11-16,2016-11-16A,46.30,46.18,0,47,0.2000,0.2000,0.0000,0.000,1.000,0.000
575,2,"The Market In 5 Minutes: DryShips, Trump And T...",A,Agilent Technologies,2016,11,16,2016-11-16,2016-11-16A,46.30,46.18,0,47,0.0000,0.0000,0.0000,0.000,1.000,0.000
576,1,Pressure Biosciences CEO On Its Beaten Down St...,A,Agilent Technologies,2016,12,6,2016-12-06,2016-12-06A,44.58,44.84,1,50,0.5963,0.2481,-0.4417,0.199,0.717,0.083


In [73]:
# ZZZ= A_ticker.groupby(['Year', 'WeekNo']).mean()
# ZZZ= ZZZ.drop(columns='index')
# ZZZ

In [101]:
#Create the feature data set
X = A_ticker[['Opening', 'Closing', 'Subjectivity', 'Polarity', 'Compound', 'Negative', 'Neutral', 'Positive']]
X = np.array(X)

In [102]:
#Create the target data set
y = np.array(A_ticker['StockChg'])

In [103]:
#Split the data into 80% training and 20% testing data sets
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 0)

In [105]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
model = LinearDiscriminantAnalysis().fit(x_train, y_train)

In [106]:
#Get the models predictions/classification
predictions = model.predict(x_test)
predictions

array([1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1,
       1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1,
       1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1,
       1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 1])

In [108]:
#Show the models metrics
from sklearn.metrics import classification_report
print( classification_report(y_test, predictions) )

              precision    recall  f1-score   support

           0       1.00      0.97      0.98        65
           1       0.96      1.00      0.98        51

    accuracy                           0.98       116
   macro avg       0.98      0.98      0.98       116
weighted avg       0.98      0.98      0.98       116



In [ ]:
withPrices_df['title']= withPrices_df['title'].astype('str')

In [ ]:
# # Extracting features from text files
# from sklearn.feature_extraction.text import CountVectorizer
# count_vect = CountVectorizer()
# X_train_counts = count_vect.fit_transform(twenty_train.data)
# X_train_counts.shape

In [ ]:
withPrices_df.dtypes

In [ ]:
withPrices_df['title'].head(6)

In [ ]:
news_vector= withPrices_df['title']
news_vector.dtypes

In [ ]:
news_vector_arr= news_vector.tolist()
news_vector_arr

In [ ]:
from sklearn.model_selection import train_test_split
X, y = withPrices_df.data, withPrices_df.target
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=37)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(news_vector_arr)   #potentially add .data to end on ['title']
X_train_counts.shape

In [ ]:
# from sklearn.model_selection import train_test_split


In [ ]:
# TF-IDF
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, twenty_train.target)